In [ ]:
# To Dos
# - Make value sets for all vaccine products
# - Rename value sets to match L3 ids
# - Deal with duplicate input options? Or maybe just let OCL take the latest one?
#
#
#
#
#
#

In [1]:
# Parameters
environment = "staging."
ocl_org = "WHO-Smart-Guidelines"
base_canonical_url = "https://smart.who.int/immunizations-measles/ValueSet/"
main_dak_source = 'who-measles-dak-dictionary'


In [2]:
import pandas as pd
import urllib.request
import numpy as np
import json

In [3]:
# import findspark
# findspark.init()
# import pyspark

In [4]:
# Dataframe with sheet title and url
urls_df = pd.DataFrame({
    'title': ['IMMZ.C Register Client', 'IMMZ.G Pre-vaccination Data', 'IMMZ.G Administer Vaccine', 'IMMZ.J Generate Reports', 'IMMZ.Z Vaccine Library'],
    'url': [
        'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=0',
        'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=536740205',
        'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=651729522',
        'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=1146884570',
        'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=975478021'
    ]
})


In [5]:
# Dictionary to store DataFrames  
df_dict = {}

# Loop through the rows in the dataframe 
for index, row in urls_df.iterrows():
    title = row['title']
    url = row['url']
    
    # Read CSV into df_dict
    df_dict[title] = pd.read_csv(url)

In [6]:
# print(df_dict.keys())
# print(df_dict['IMMZ.C Register Client'].head())
# print(df_dict['IMMZ.G Pre-vaccination Data'].head())

In [6]:
# Concatenate all DataFrames 
df = pd.concat(df_dict.values(), keys=df_dict.keys(), verify_integrity=True)

# Add source column indicating which DataFrame it came from
df['Origin Tab'] = df.index.get_level_values(0)

# Reset index to move source out of index
df = df.reset_index(level=0, drop=True)

# Drop unnamed columns
df = df.drop(columns=[col for col in df.columns if 'unnamed' in col.lower()])

# Fix empty values
df = df.fillna('')
df = df.astype(str)

  # Combine duplicative columns
df['ICD-11 Comments / Considerations'] = df.apply(lambda row: row['ICD-11 Comments / Considerations'] if row['ICD-11 Comments / Considerations'] != '' and row['ICD-11 Comments / Considerations.1'] == ''
                                                  else row['ICD-11 Comments / Considerations.1'] if row['ICD-11 Comments / Considerations'] == '' and row['ICD-11 Comments / Considerations.1'] != ''
                                                  else row['ICD-11 Comments / Considerations'] if row['ICD-11 Comments / Considerations'] == row['ICD-11 Comments / Considerations.1']
                                                  else row['ICD-11 Comments / Considerations'] + '; ' + row['ICD-11 Comments / Considerations.1'], axis=1)

# # icd11_code_cols = ['ICD-11 Code','ICD-11 Code.1']  
# icd11_comment_cols = ['ICD-11 Comments / Considerations','ICD-11 Comments / Considerations.1']

# #   # Combine base column with duplicate 
# # df['ICD-11 Code'] = df[icd11_code_cols].sum(axis=1)  
# df['ICD-11 Comments / Considerations'] = df[icd11_comment_cols].sum(axis=1)

#   # Drop duplicate columns
# df.drop(columns=icd11_code_cols[1], inplace=True) 
df.drop(columns='ICD-11 Comments / Considerations.1', inplace=True)

# Fix unencoded values
# df = df.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
#df.replace({'â€“': '-'})
df = df.replace('–', '-', regex=True)

# Export combined DataFrame to a CSV
df.to_csv('combined.csv', index=False, encoding='utf-8')

#print(df.head())

In [7]:
# Transforming CSV for OCL import

# Define a dictionary mapping old column names to new names
col_rename_map = {
  "Activity ID": "attr:Activity_ID",
  "Data Element ID": "id",
  "Data Element Label": "name[1]",
  "Description and Definition": "description[1]",
  "Multiple Choice Type (if applicable)": "attr:Multiple_Choice_Type_(if_applicable)",
  "Data Type": "datatype",
  "Calculation": "attr:Calculation",
  "Quantity Sub-Type": "attr:Quantity_Sub-Type",
  "Validation Condition": "attr:Validation_Condition",
  "Editable": "attr:Editable",
  "Required": "attr:Required",
  "Skip Logic": "attr:Skip_Logic",
  "Linkages to Aggregate Indicators": "attr:Linkages_to_Aggregate_Indicators",
  "Notes": "attr:Notes",
  "ICD-11 URI": "attr:ICD-11_URI",
  "ICD-11 Comments / Considerations": "attr:ICD-11_Comments_Considerations",
  "ICD-11 Relationship": "map_type[0]",
  "ICD-10 Comments / Considerations": "attr:ICD-10_Comments_Considerations",
  "ICD-10 Relationship": "map_type[1]",
  "ICD-9 Comments / Considerations": "attr:ICD-9_Comments_Considerations",
  "ICD-9 Relationship": "map_type[2]",
  "LOINC version 2.68 Comments / Considerations": "attr:LOINC_version_2.68_Comments_Considerations",
  "LOINC version 2.68 Relationship": "map_type[3]",
  "ICHI URI": "attr:ICHI_URI",
  "ICHI Comments / Considerations": "attr:ICHI_Comments_Considerations",
  "ICHI Relationship": "map_type[4]",
  "ICF Comments / Considerations": "attr:ICF_Comments_Considerations",
  "ICF Relationship": "map_type[5]",
  "SNOMED GPS Code Comments Considerations": "attr:SNOMED_GPS_Code_Comments_Considerations",
  "SNOMED GPS Relationship": "map_type[6]",
  "SNOMED CT International Version Comments / Considerations": "attr:Snomed_CT_International_Version_Comments_Considerations",
  "SNOMED CT Relationship": "map_type[7]",
  "HL7 FHIR R4 - Resource": "attr:HL7_FHIR_R4_Resource",
  "HL7 FHIR R4 - Values": "attr:HL7_FHIR_R4_Values",
  "HL7 FHIR R4 Relationship": "map_type[8]",
  "ICD-11 Description": "attr:ICD-11_Description",
  "WHO ATC Name": "attr:WHO_ATC_Name",
  "Origin Tab": "attr:Activity_Group",
  "Input Options": "attr:Input_Options",
  "ICD-11 Code": "map_to_concept_url[0]",
  "ICD-10 Code": "map_to_concept_url[1]",
  "ICD-9 Code": "map_to_concept_url[2]",
  "LOINC version 2.68 Code": "map_to_concept_url[3]",
  "ICHI (Beta 3) Code": "map_to_concept_url[4]",
  "ICF Code": "map_to_concept_url[5]",
  "SNOMED GPS Code": "map_to_concept_url[6]",
  "SNOMED CT International Version Code": "map_to_concept_url[7]",
  "HL7 FHIR R4 Code": "map_to_concept_url[8]",
  "WHO ATC Code": "map_to_concept_url[9]"
}

# Rename columns using the mapping
df = df.rename(columns=col_rename_map)

# Set 'attr:Input_Options' to empty if same as 'Data Element ID'
df['attr:Input_Options'] = df.apply(lambda row: '' if row['attr:Multiple_Choice_Type_(if_applicable)'] == "Input Option" else row['attr:Input_Options'], axis=1)
#print(df.head())

In [8]:
# Prepping mappings for OCL CSV

# Mapping prefixes (Column, prefix)
prefix_dict = {
  "map_to_concept_url[0]": "/orgs/WHO/sources/ICD-11-WHO/concepts/",
  "map_to_concept_url[1]": "/orgs/WHO/sources/ICD-10-WHO/concepts/",
  "map_to_concept_url[2]": "/orgs/WHO/sources/ICD-9-WHO/concepts/",
  "map_to_concept_url[3]": "/orgs/Regenstrief/sources/LOINC/concepts/",
  "map_to_concept_url[4]": "/orgs/WHO/sources/WHO-ICHI/concepts/",
  "map_to_concept_url[5]": "/orgs/WHO/sources/WHO-ICF/concepts/",
  "map_to_concept_url[6]": "/orgs/SNOMED-International/sources/SNOMED-GPS/concepts/",
  "map_to_concept_url[7]": "/orgs/IHTSDO/sources/SNOMED-CT/concepts/",
  "map_to_concept_url[9]": "/orgs/WHO/sources/WHOATC/concepts/"
}

# For each column col:
#     It checks if the column exists in the DataFrame using if col in df.columns.
#     If the column exists, it applies a lambda function to the column using df[col].apply(lambda x: ...).
#     The lambda function checks if the string 'Not classifiable' is present in the value x. If it is, it replaces the value with an empty string ''.
#     If 'Not classifiable' is not present, it splits the value x using the space character ' ' and takes the first part str(x).split(' ')[0].

for col, prefix in prefix_dict.items():
    # Check if column exists in the DataFrame
    if col in df.columns:
        df[col] = df[col].apply(lambda x: '' if 'Not classifiable' in str(x) else (prefix + str(x).split(' ')[0] if str(x) else ''))

        # Convert map types to match OCL? Nah maybe later.
        
### Deal with HL7 codes to tie them to a code system

# Dictionary with column value and system URL
url_dict = {
    "male": "/orgs/HL7/sources/administrative-gender/",
    "female": "/orgs/HL7/sources/administrative-gender/",
    "other": "/orgs/HL7/sources/administrative-gender/",
    "unknown": "/orgs/HL7/sources/administrative-gender/",
    "http://hl7.org/fhir/uv/ips/ValueSet/vaccines-gps-uv-ips": "http://hl7.org/fhir/uv/ips/ValueSet/vaccines-gps-uv-ips",
    "http://hl7.org/fhir/uv/ips/ValueSet/whoatc-uv-ips": "http://hl7.org/fhir/uv/ips/ValueSet/whoatc-uv-ips",
    "http://hl7.org/fhir/ValueSet/immunization-route": "http://hl7.org/fhir/ValueSet/immunization-route",
    "complete | pending | error": "https://build.fhir.org/valueset-measure-report-status.html",
    "summary": "https://build.fhir.org/valueset-measure-report-type.html",
    "increase": "/orgs/fhir-hl7-test/sources/measure-improvement-notation/",
    "decrease": "/orgs/fhir-hl7-test/sources/measure-improvement-notation/",
    "numerator": "/orgs/fhir-hl7-test/sources/measure-population/",
    "denominator": "/orgs/fhir-hl7-test/sources/measure-population/"
}

# Update the column 'map_to_concept_url[8]'
df['map_to_concept_url[8]'] = df.apply(lambda row: '' if 'Not classifiable' in str(row['map_to_concept_url[8]']) else
                                      (url_dict.get(row['map_to_concept_url[8]'], '') + 'concepts/' + row['map_to_concept_url[8]']) if '/orgs/' in url_dict.get(row['map_to_concept_url[8]'], '') else
                                      url_dict.get(row['map_to_concept_url[8]'], ''), axis=1)

# Fixes blank map types for HL7 FHIR concepts and assigns them as "Related to"
df.loc[(df['map_to_concept_url[8]'] != '') & (df['map_type[8]'] == ''), 'map_type[8]'] = 'Related to'

# Create map_type[9] column based on map_to_concept_url[9]
map_to_col_9 = "map_to_concept_url[9]"
map_type_col_9 = "map_type[9]"

# Makes a mapping column for WHO ATC and sets them to "Equivalent" (which applies only to WHO ATC drug codes) 
df[map_type_col_9] = ""
df.loc[df[map_to_col_9].notnull(), map_type_col_9] = "Equivalent"

In [9]:
# Misc Cleanup

# Assign Map Type if mapping is present
# Iterate through each column of map_to_concept_url and map_type
for i in range(10):
    # Update map_type based on map_to_concept_url
    df.loc[(df[f'map_to_concept_url[{i}]'] != '') & (df[f'map_type[{i}]'] == ''), f'map_type[{i}]'] = 'Unspecified map type'

# Remove Map Type if no mapping is present  
for i in range(10):
    map_to_col = f"map_to_concept_url[{i}]"
    map_type_col = f"map_type[{i}]"
    
    df.loc[df[map_to_col].apply(lambda x: x.strip() == '') | df[map_to_col].isnull(), map_type_col] = "" 

In [10]:
# Create OCL-required columns that don't have special logic
df['resource_type'] = 'Concept'
df['source'] = main_dak_source
df['owner_id'] = ocl_org
df['owner_type'] = 'Organization'
df['retired'] = 'FALSE'
df['name_type[1]'] = 'Fully Specified'

In [11]:
# Assign concept IDs for input options and other values without IDs (addressed below)
# df.loc[df['id'] == '', 'id'] = range(1, 1 + len(df[df['id'] == '']))

# Relate input options back to their respective question

    # Store the existing 'id' values in a new column 'map_from_concept_url[10]', which will serve as the "question" for q-and-a mappings
df['map_from_concept_url[10]'] = df['id'].astype(str)
df['map_type[10]'] = "Q-AND-A"

    # Generate numerical IDs for rows without an existing 'id' value, then create Q-and-A mappings for those rows to connect them back to the data element (this step is dependent on spreadsheet order) 
df.loc[df['id'] == '', 'id'] = range(1, 1 + len(df[df['id'] == '']))
df['map_to_concept_url[10]'] = "/orgs/" + df['owner_id'].astype(str) + "/sources/" + df['source'].astype(str) + "/concepts/" + df['id'].astype(str) + "/"
df['map_from_concept_url[10]'] = "/orgs/" + df['owner_id'].astype(str) + "/sources/" + df['source'].astype(str) + "/concepts/" + df['map_from_concept_url[10]'].replace('', method='ffill') + "/"
df.loc[df['map_from_concept_url[10]'] == "/orgs/" + df['owner_id'].astype(str) + "/sources/" + df['source'].astype(str) + "/concepts/" + df['id'].astype(str) + "/", ['map_type[10]', 'map_from_concept_url[10]', 'map_to_concept_url[10]']] = ''

df['concept_class'] = df.apply(lambda x: 'Input Option' if x['map_type[10]'] != '' else 'Data Element', axis=1)


In [16]:
# Deal with duplicate input options? Or maybe just let OCL take the latest one?
# merged_df.head()

,id,name,full_name,owner,owner_type,default_locale,canonical_url,collection_type,type,__action
0,IMMZ.C1.DE5-values,IMMZ.C1.DE5-values: Sex,Values for: Sex,WHO-Smart-Guidelines,Organization,en,https://smart.who.int/immunizations-measles/Va...,Value Set,Collection,DELETE
1,IMMZ.E0.DE10-values,IMMZ.E0.DE10-values: HIV status,Values for: HIV status,WHO-Smart-Guidelines,Organization,en,https://smart.who.int/immunizations-measles/Va...,Value Set,Collection,DELETE
2,IMMZ.E0.DE15-values,IMMZ.E0.DE15-values: TST Test Result,Values for: TST Test Result,WHO-Smart-Guidelines,Organization,en,https://smart.who.int/immunizations-measles/Va...,Value Set,Collection,DELETE
3,IMMZ.E0.DE16-values,IMMZ.E0.DE16-values: IGRA Test Result,Values for: IGRA Test Result,WHO-Smart-Guidelines,Organization,en,https://smart.who.int/immunizations-measles/Va...,Value Set,Collection,DELETE
4,IMMZ.E0.DE18-values,IMMZ.E0.DE18-values: Maternal HIV status,Values for: Maternal HIV status,WHO-Smart-Guidelines,Organization,en,https://smart.who.int/immunizations-measles/Va...,Value Set,Collection,DELETE


In [15]:
### Make df_v, which is a list of value sets (made of input options) to be loaded into OCL

# Copies dataframe and subsets only to unique values that belong in valuesets
df_v = df[df['map_from_concept_url[10]'] != ''].drop_duplicates(subset=['map_from_concept_url[10]']).copy()
df_v = df_v[['map_from_concept_url[10]']].copy()
df_v['id'] = df_v['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]
df_v['lookup'] = df_v['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]
df_v = df_v.drop_duplicates(subset='id').copy()

# Merge the two dataframes based on the 'id' column
merged_df = pd.merge(df_v, df, left_on='lookup', right_on='id', how='left')

    # Access the value of the matching 'id' from the 'df' dataframe
matching_value = merged_df['name[1]'].values[0]  # Retrieves the first matching value

    # Keep only the specified columns and rename them
merged_df = merged_df[['id_x', 'name[1]']].rename(columns={'id_x': 'id', 'name[1]': 'name'})

# Change/update columns for Bulk Import
merged_df['full_name'] = 'Values for: ' + merged_df['name']
merged_df['owner'] = ocl_org
merged_df['owner_type'] = "Organization"
merged_df['default_locale'] = "en"
merged_df['canonical_url'] = base_canonical_url + merged_df['id']
merged_df['collection_type'] = "Value Set"
merged_df['type'] = "Collection"
merged_df['id'] = merged_df['id']+"-values"
merged_df['name'] = merged_df['id']+": "+ merged_df['name']


# Outputs the value sets to a JSON file
with open('ocl-import-measles-value-sets.json', 'w') as outfile:
    for index, row in merged_df.iterrows():
        json.dump(row.to_dict(), outfile, ensure_ascii=False)
        outfile.write('\n')

# Delete value sets if needed
merged_df['__action'] = "DELETE"

with open('delete-ocl-import-measles-value-sets.json', 'w') as outfile:
    for index, row in merged_df.iterrows():
        json.dump(row.to_dict(), outfile, ensure_ascii=False)
        outfile.write('\n')

In [17]:
###### To Do!
### Makes value set list for Immunization Library

# If there are NaN values, replace them with a suitable value
df['id'] = df['id'].fillna('') # Replace NaN with an empty string
    
# df.head()
# Filter the DataFrame to only include rows where the 'id' column starts with 'IMMZ.Z1'
#filtered_df = df[df['id'].str.startswith('IMMZ.Z1')]





In [18]:
### Adds ValueSet URL to df

#Example: https://app.staging.openconceptlab.org/#/orgs/WHO-Smart-Guidelines/collections/IMMZ.E0.DE43-values/
base_url = "https://app."+environment+"openconceptlab.org/#/orgs/WHO-Smart-Guidelines/collections/"

# Create a temporary DataFrame with only the 'id' column from merged_df
temp_df = merged_df[['id']].copy()

# Create a new 'matching_ID' column by removing "-values" from the 'id' column
temp_df['matching_ID'] = temp_df['id'].str.replace('-values', '')

# Merge df with temp_df on 'id' and 'matching_ID' to identify matching IDs
merged_ids = df.merge(temp_df, left_on='id', right_on='matching_ID', how='inner')

# Create 'attr:ValueSet_URL' column based on matching IDs
df['attr:ValueSet_URL'] = df['id'].apply(lambda x: x + '-values' if x in merged_ids['matching_ID'].values else '')
# Concatenate base_url to the existing values in 'attr:ValueSet_URL' column
df['attr:ValueSet_URL'] = df.apply(lambda row: base_url + row['attr:ValueSet_URL'] if row['attr:ValueSet_URL'] else '', axis=1)

In [19]:
# Output as OCL-formatted CSV
df.to_csv('ocl-import-measles-dak.csv', index=False, encoding='utf-8')

# Output test OCL file with only 10 rows
#df.head(10).to_csv('SHORT - ocl-import-measles-dak.csv', index=False, encoding='utf-8')

# Output value set list with only 10 rows (not needed now that it's in JSON format)
#merged_df.head(10).to_csv('SHORT - ocl-import-measles-value-sets.csv', index=False, encoding='utf-8')

In [20]:
### Makes df of references to import into the respective collections

# Copies dataframe and subsets only to unique values that belong in valuesets
df_ref = df[df['map_from_concept_url[10]'] != ''].drop_duplicates(subset=['map_from_concept_url[10]','map_to_concept_url[10]']).copy()
df_ref = df_ref[['map_from_concept_url[10]','map_to_concept_url[10]']].copy()
df_ref['col_id'] = df_ref['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]+"-values"
df_ref['collection_url'] = "/orgs/" + ocl_org +"/collections/" + df_ref['col_id'] + "/"
df_ref = df_ref[['collection_url', 'map_to_concept_url[10]']].rename(columns={'map_to_concept_url[10]': 'data:expressions'})
df_ref["resource_type"] = "Reference"

#df_ref.head(10).to_csv('SHORT - ocl-import-measles-value-set-references.csv', index=False, encoding='utf-8')
#display(df_ref)
df_ref["type"] = "Reference"
output_json = df_ref.apply(lambda x: json.dumps({"type": x['type'], "collection_url": x['collection_url'], "data": {"expressions": [x['data:expressions']]}}), axis=1)

# Print or save the JSON output
# for item in output_json:
#     print(item)

#print(output_json)

# Write the output to a JSON file
with open("ocl-import-measles-value-set-references.json", "w") as outfile:
    for line in output_json:
        outfile.write(line + '\n')
# Short JSON output
# with open("SHORT - ocl-import-measles-value-set-references.json", "w") as outfile:
#     for line in output_json[:9]:
#         outfile.write(line + '\n')


In [23]:
### Make df_vs_vers, which contains lines to create collection versions for the various value sets
# Create a new DataFrame "df_vs_vers" with selected columns from "merged_df" and renamed column
df_vs_vers = merged_df[['owner', 'id']].copy()
df_vs_vers.rename(columns={'id': 'collection','owner': 'owner_id'}, inplace=True)

# Add new columns "resource_type", "id", and "description" with specified values
df_vs_vers['resource_type'] = 'Collection Version'
df_vs_vers['id'] = '0.1.0'
df_vs_vers['description'] = 'Initial Load'

# Display the updated DataFrame "df_vs_vers"
#print(df_vs_vers)

#Output as CSV
#df_vs_vers.to_json('ocl-import-measles-value-set-versions.json', orient='records', lines=True)
df_vs_vers.to_csv('ocl-import-measles-value-set-versions.csv', index=False, encoding='utf-8')